In [1]:
import pandas as pd
import numpy as np

df = pd.read_excel("Opportunity Details.xlsx", engine="openpyxl")
df.head()

,Index,-SUM([Annl Agg Savings - treated]),Customer,Super Payer Short,Payer Short,Solution,LOB,Product Type,Claim Type,Medical Policy,...,Dp Desc,DP Age in Years,Disposition,Disposition Date,Cpw Reason,Decision Status,Decision Date,Model Recommendation,Annl Edits,Annl Agg Savings
0,Grand Total,Total,Total,Total,Total,Total,Total,Total,Total,Total,...,Total,Total,Total,Total,Total,Total,Total,Total,233347.737883,1.098342e+08
1,1,"-19,178,173.6477395",Highmark,HIGAL,HIGPB,PPM,BlueCard,ICM,P: Prof,Drug and Biological Policy Processing and Poli...,...,Deny a drug supplied in only single-dose formu...,2,Invalid,Aug-2025,"[""Inappropriate for Line of Business""]",Reject,Oct-2024,Not Recommended,30300.719754,1.917817e+07
2,2,"-10,089,590.0464801",Highmark,HIGAL,HIGPA,PPM,Commercial,ICM,P: Prof,Drug and Biological Policy Processing and Poli...,...,Deny a drug supplied in only single-dose formu...,2,No Disposition,Jan-2026,NaN,Reject,Oct-2024,Not Recommended,19547.879864,1.008959e+07
3,3,"-5,520,343.93132299",Highmark,HIGAL,HIGAC,PPM,Commercial,ICM,P: Prof,Drug and Biological Policy Processing and Poli...,...,Deny a drug supplied in only single-dose formu...,2,No Disposition,Jan-2026,NaN,Reject,Oct-2024,Not Recommended,8702.519826,5.520344e+06
4,4,"-4,556,194.00204734",Highmark,HIGAL,HIGPB,PPM,BlueCard,ICMO,O: OP Facility,Drug and Biological Policy Processing and Poli...,...,Deny a drug supplied in only single-dose formu...,2,Invalid,Aug-2025,"[""Inappropriate for Line of Business""]",Reject,Oct-2024,Not Recommended,3665.440025,4.556194e+06


In [2]:
# STEP 2: remove the "Grand Total" row, clean columns, and join PolicyCollection + PolicyGroup from mapping file

# 1) Drop the total row (keep header as-is, start data from original Index 1)
df = df.iloc[1:].reset_index(drop=True)

# 2) Drop unwanted columns
df = df.drop(columns=["Index", "-SUM([Annl Agg Savings - treated])"], errors="ignore")

# 3) Join PolicyCollection + PolicyGroup from Policy Collection Mapping (by DP Key)
df_map = pd.read_excel(
    "Policy Collection Mapping.xlsx",
    engine="openpyxl",
    usecols=["DPKey", "PolicyCollection", "PolicyGroup"]
)

df_map["DPKey"] = pd.to_numeric(df_map["DPKey"], errors="coerce")
df["Dp Key"] = pd.to_numeric(df["Dp Key"], errors="coerce")

df = (
    df.merge(df_map.drop_duplicates("DPKey"), how="left", left_on="Dp Key", right_on="DPKey")
      .drop(columns=["DPKey"])
      .rename(columns={"PolicyCollection": "Policy Collection", "PolicyGroup": "Policy Group"})
)


In [3]:
# quick view to confirm
print(df.shape)
print(df.columns.tolist())
df.head(5)

(1550, 22)
['Customer', 'Super Payer Short', 'Payer Short', 'Solution', 'LOB', 'Product Type', 'Claim Type', 'Medical Policy', 'Topic', 'Dp Key', 'Dp Desc', 'DP Age in Years', 'Disposition', 'Disposition Date  ', 'Cpw Reason', 'Decision Status', 'Decision Date', 'Model Recommendation', 'Annl Edits', 'Annl Agg Savings', 'Policy Collection', 'Policy Group']


,Customer,Super Payer Short,Payer Short,Solution,LOB,Product Type,Claim Type,Medical Policy,Topic,Dp Key,...,Disposition,Disposition Date,Cpw Reason,Decision Status,Decision Date,Model Recommendation,Annl Edits,Annl Agg Savings,Policy Collection,Policy Group
0,Highmark,HIGAL,HIGPB,PPM,BlueCard,ICM,P: Prof,Drug and Biological Policy Processing and Poli...,Drug Wastage Modifiers JW and JZ,17227,...,Invalid,Aug-2025,"[""Inappropriate for Line of Business""]",Reject,Oct-2024,Not Recommended,30300.719754,1.917817e+07,Drug wastage,Drug Wastage
1,Highmark,HIGAL,HIGPA,PPM,Commercial,ICM,P: Prof,Drug and Biological Policy Processing and Poli...,Drug Wastage Modifiers JW and JZ,17227,...,No Disposition,Jan-2026,NaN,Reject,Oct-2024,Not Recommended,19547.879864,1.008959e+07,Drug wastage,Drug Wastage
2,Highmark,HIGAL,HIGAC,PPM,Commercial,ICM,P: Prof,Drug and Biological Policy Processing and Poli...,Drug Wastage Modifiers JW and JZ,17227,...,No Disposition,Jan-2026,NaN,Reject,Oct-2024,Not Recommended,8702.519826,5.520344e+06,Drug wastage,Drug Wastage
3,Highmark,HIGAL,HIGPB,PPM,BlueCard,ICMO,O: OP Facility,Drug and Biological Policy Processing and Poli...,Drug Wastage Modifiers JW and JZ,17227,...,Invalid,Aug-2025,"[""Inappropriate for Line of Business""]",Reject,Oct-2024,Not Recommended,3665.440025,4.556194e+06,Drug wastage,Drug Wastage
4,Highmark,GTWHP,GTWPA,PPM,Medicaid,ICMO,O: OP Facility,Drug and Biological Policy Processing and Poli...,Drug Wastage Modifiers JW and JZ,17227,...,No Disposition,Jun-2025,[],Approve Library,Dec-2025,Less Likely,3300.119966,4.553132e+06,Drug wastage,Drug Wastage


In [4]:
# STEP 3: Format Annl Edits + Annl Agg Savings as whole numbers and display

# Annl Edits -> whole number
if "Annl Edits" in df.columns:
    df["Annl Edits"] = pd.to_numeric(df["Annl Edits"], errors="coerce").round(0).astype("Int64")

# Annl Agg Savings -> whole number (no decimals)
if "Annl Agg Savings" in df.columns:
    df["Annl Agg Savings"] = pd.to_numeric(df["Annl Agg Savings"], errors="coerce").round(0).astype("Int64")

# Display formatting (no scientific notation, no decimals)
pd.set_option("display.float_format", lambda x: f"{x:,.0f}")

df.head(10)

,Customer,Super Payer Short,Payer Short,Solution,LOB,Product Type,Claim Type,Medical Policy,Topic,Dp Key,...,Disposition,Disposition Date,Cpw Reason,Decision Status,Decision Date,Model Recommendation,Annl Edits,Annl Agg Savings,Policy Collection,Policy Group
0,Highmark,HIGAL,HIGPB,PPM,BlueCard,ICM,P: Prof,Drug and Biological Policy Processing and Poli...,Drug Wastage Modifiers JW and JZ,17227,...,Invalid,Aug-2025,"[""Inappropriate for Line of Business""]",Reject,Oct-2024,Not Recommended,30301,19178174,Drug wastage,Drug Wastage
1,Highmark,HIGAL,HIGPA,PPM,Commercial,ICM,P: Prof,Drug and Biological Policy Processing and Poli...,Drug Wastage Modifiers JW and JZ,17227,...,No Disposition,Jan-2026,NaN,Reject,Oct-2024,Not Recommended,19548,10089590,Drug wastage,Drug Wastage
2,Highmark,HIGAL,HIGAC,PPM,Commercial,ICM,P: Prof,Drug and Biological Policy Processing and Poli...,Drug Wastage Modifiers JW and JZ,17227,...,No Disposition,Jan-2026,NaN,Reject,Oct-2024,Not Recommended,8703,5520344,Drug wastage,Drug Wastage
3,Highmark,HIGAL,HIGPB,PPM,BlueCard,ICMO,O: OP Facility,Drug and Biological Policy Processing and Poli...,Drug Wastage Modifiers JW and JZ,17227,...,Invalid,Aug-2025,"[""Inappropriate for Line of Business""]",Reject,Oct-2024,Not Recommended,3665,4556194,Drug wastage,Drug Wastage
4,Highmark,GTWHP,GTWPA,PPM,Medicaid,ICMO,O: OP Facility,Drug and Biological Policy Processing and Poli...,Drug Wastage Modifiers JW and JZ,17227,...,No Disposition,Jun-2025,[],Approve Library,Dec-2025,Less Likely,3300,4553132,Drug wastage,Drug Wastage
5,Highmark,HIGAL,HIGPF,PPM,Federal Employee Program,ICM,P: Prof,Drug and Biological Policy Processing and Poli...,Drug Wastage Modifiers JW and JZ,17227,...,Present,Jul-2025,"[""Business Reasons""]",NaN,Never Presented,Less Likely,3856,3214441,Drug wastage,Drug Wastage
6,Highmark,HIGAL,HIGPA,PPM,Commercial,ICMO,O: OP Facility,Drug and Biological Policy Processing and Poli...,Drug Wastage Modifiers JW and JZ,17227,...,No Disposition,Jan-2026,NaN,Reject,Oct-2024,Not Recommended,2270,3014905,Drug wastage,Drug Wastage
7,Highmark,HIGAL,HIGNY,PPM,Commercial,ICM,P: Prof,Drug and Biological Policy Processing and Poli...,Drug Wastage Modifiers JW and JZ,17227,...,No Disposition,Dec-2025,NaN,Reject,Oct-2024,Not Recommended,3868,2391077,Drug wastage,Drug Wastage
8,Highmark,HIGAL,HIGAC,PPM,Commercial,ICMO,O: OP Facility,Drug and Biological Policy Processing and Poli...,Drug Wastage Modifiers JW and JZ,17227,...,No Disposition,Jan-2026,NaN,Reject,Oct-2024,Not Recommended,1039,1903379,Drug wastage,Drug Wastage
9,Highmark,HIGAL,HIGPA,PPM,Commercial,ICMO,O: OP Facility,Drug & Biological Policy,"Immune Globulins, IV (90283, J1459, J1552, J15...",7850,...,Invalid,Jul-2025,"[""Performed by Another Vendor""]",Reject,Dec-2022,Not Recommended,697,1515134,All indications,Indications


In [5]:
# =========================
# STEP (Before Step 4): Read CPW Disp_CDM Dec Detail view + Join into df_cpw
# (Uses openpyxl directly to avoid XLRDError with pandas==0.25.3)
# =========================

import pandas as pd
from openpyxl import load_workbook

# ---- 1) Path / Sheet settings ----
CPW_FILE_PATH = r"CPW Disp_CDM Dec Detail view.xlsx"   # <-- update if needed
CPW_SHEET_NAME = 0  # int index (0 = first sheet) OR sheet name string

# ---- 2) Read CPW excel using openpyxl (bypasses pandas->xlrd for xlsx) ----
wb = load_workbook(CPW_FILE_PATH, data_only=True)

if isinstance(CPW_SHEET_NAME, int):
    ws = wb.worksheets[CPW_SHEET_NAME]
else:
    ws = wb[CPW_SHEET_NAME]

rows = list(ws.values)
if not rows or len(rows) < 2:
    raise ValueError("CPW sheet appears empty or missing a header row.")

header = [str(x).strip() if x is not None else "" for x in rows[0]]
data = rows[1:]

df_cpw = pd.DataFrame(data, columns=header)

# Drop columns with blank header names (pandas 0.25 safe)
keep_cols_mask = [str(c).strip() != "" for c in df_cpw.columns]
df_cpw = df_cpw.loc[:, keep_cols_mask]

# ---- 3) Columns to bring from CPW ----
cpw_cols_to_add = [
    "CDM Dec Dt.",
    "CDM Dec Status",
    "Change Mods",
    "CPW Client Mods",
    "CPW Disp",
    "CPW Reason",
    "Disposition Capt Date",
]

# ---- 4) Validate required columns exist in CPW ----
cpw_required = ["Payer Short", "DP Key", "LOB Desc"] + cpw_cols_to_add
missing_cpw = [c for c in cpw_required if c not in df_cpw.columns]
if missing_cpw:
    raise ValueError(
        "Missing required column(s) in CPW file:\n"
        + "\n".join(missing_cpw)
        + "\n\nAvailable columns:\n"
        + ", ".join(df_cpw.columns.astype(str))
    )

# ---- 5) Prepare join keys (trim + uppercase for string matching) ----
def _norm_str(s):
    return s.astype(str).str.strip().str.upper()

# Left table is your opportunity dataframe (use df or df_f depending on where you are in the notebook)
LEFT_DF = df  # <-- change to df_f if you want to join after filters

LEFT_DF = LEFT_DF.copy()
df_cpw = df_cpw.copy()

LEFT_DF["__ps"] = _norm_str(LEFT_DF["Payer Short"])
LEFT_DF["__dp"] = _norm_str(LEFT_DF["Dp Key"])
LEFT_DF["__lob"] = _norm_str(LEFT_DF["LOB"])

df_cpw["__ps"] = _norm_str(df_cpw["Payer Short"])
df_cpw["__dp"] = _norm_str(df_cpw["DP Key"])
df_cpw["__lob"] = _norm_str(df_cpw["LOB Desc"])

# ---- 6) De-duplicate CPW to avoid exploding rows on merge ----
df_cpw["__disp_capt_dt"] = pd.to_datetime(df_cpw["Disposition Capt Date"], errors="coerce")

df_cpw_sorted = df_cpw.sort_values(
    ["__ps", "__dp", "__lob", "__disp_capt_dt"],
    ascending=[True, True, True, True]
)

df_cpw_latest = (
    df_cpw_sorted.groupby(["__ps", "__dp", "__lob"], as_index=False)
    .tail(1)
)

df_cpw_latest = df_cpw_latest[["__ps", "__dp", "__lob"] + cpw_cols_to_add].copy()

# ---- 7) Merge ----
df_merged = LEFT_DF.merge(
    df_cpw_latest,
    how="left",
    on=["__ps", "__dp", "__lob"],
    suffixes=("", "_cpw"),
)

# ---- 8) Cleanup helper columns ----
df_merged.drop(columns=["__ps", "__dp", "__lob"], inplace=True, errors="ignore")

# optional cleanup of temp cols in intermediate dfs
for _tmp in [df_cpw, df_cpw_latest]:
    _tmp.drop(columns=["__ps", "__dp", "__lob"], inplace=True, errors="ignore")
df_cpw.drop(columns=["__disp_capt_dt"], inplace=True, errors="ignore")

# ---- 9) Sanity check ----
print("Rows (before merge):", len(LEFT_DF))
print("Rows (after merge): ", len(df_merged))
print("CPW columns added:", cpw_cols_to_add)

# ---- 10) Assign output to df_cpw ----
df_cpw = df_merged

df_cpw.head(5)


Rows (before merge): 1550
Rows (after merge):  1550
CPW columns added: ['CDM Dec Dt.', 'CDM Dec Status', 'Change Mods', 'CPW Client Mods', 'CPW Disp', 'CPW Reason', 'Disposition Capt Date']


,Customer,Super Payer Short,Payer Short,Solution,LOB,Product Type,Claim Type,Medical Policy,Topic,Dp Key,...,Annl Agg Savings,Policy Collection,Policy Group,CDM Dec Dt.,CDM Dec Status,Change Mods,CPW Client Mods,CPW Disp,CPW Reason,Disposition Capt Date
0,Highmark,HIGAL,HIGPB,PPM,BlueCard,ICM,P: Prof,Drug and Biological Policy Processing and Poli...,Drug Wastage Modifiers JW and JZ,17227,...,19178174,Drug wastage,Drug Wastage,"October 4, 2024",Reject,,10/2024: Turned off due to provider abrasion....,Invalid,"[""Inappropriate for Line of Business""]",2025-08-13
1,Highmark,HIGAL,HIGPA,PPM,Commercial,ICM,P: Prof,Drug and Biological Policy Processing and Poli...,Drug Wastage Modifiers JW and JZ,17227,...,10089590,Drug wastage,Drug Wastage,"October 4, 2024",Reject,,10/2024: Turned off due to provider abrasion....,No Disposition,,2026-01-12
2,Highmark,HIGAL,HIGAC,PPM,Commercial,ICM,P: Prof,Drug and Biological Policy Processing and Poli...,Drug Wastage Modifiers JW and JZ,17227,...,5520344,Drug wastage,Drug Wastage,"October 4, 2024",Reject,,10/2024: Turned off due to provider abrasion....,No Disposition,,2026-01-12
3,Highmark,HIGAL,HIGPB,PPM,BlueCard,ICMO,O: OP Facility,Drug and Biological Policy Processing and Poli...,Drug Wastage Modifiers JW and JZ,17227,...,4556194,Drug wastage,Drug Wastage,"October 4, 2024",Reject,,10/2024: Turned off due to provider abrasion....,Invalid,"[""Inappropriate for Line of Business""]",2025-08-13
4,Highmark,GTWHP,GTWPA,PPM,Medicaid,ICMO,O: OP Facility,Drug and Biological Policy Processing and Poli...,Drug Wastage Modifiers JW and JZ,17227,...,4553132,Drug wastage,Drug Wastage,"December 22, 2025",Approve Library,,01/2026: Client approved adding Medicaid LOB f...,No Disposition,[],2025-06-12


In [6]:
import pandas as pd

# =========================
# STEP 4: Define filter dropdown options + apply filters + create df_f + pivot
# =========================

# Use the enriched CPW-joined dataframe as the base (change to df if you want original)
base_df = df_cpw.copy()

# ---- 1) Filter columns you want to expose in dropdowns ----
# NOTE: "Disposition" here maps to CPW column "CPW Disp" (adjust if your real column name differs)
FILTER_COLS = ["Disposition", "Recently presented", "LOB", "Decision Status", "Decision Date"]

# map friendly filter names -> actual dataframe column names
FILTER_COL_MAP = {
    "Disposition": "Disposition",                 # <-- adjust if needed
    "Recently presented": "Recently presented",
    "LOB": "LOB",
    "Decision Status": "Decision Status",
    "Decision Date": "Decision Date",
}

# ---- 2) Build dropdown options (unique values for each filter column) ----
filter_options = {}
for friendly_col in FILTER_COLS:
    actual_col = FILTER_COL_MAP.get(friendly_col, friendly_col)
    if actual_col in base_df.columns:
        s = base_df[actual_col].dropna().astype(str).str.strip()
        s = s.loc[s.ne("")]  # remove blanks
        filter_options[friendly_col] = sorted(s.unique().tolist())

# ---- 3) Choose filters here (single value OR list of values). Use "All" or None to skip a filter. ----
selected_filters = {
    "Disposition": ["Invalid", "No Disposition"],   # example multi-select on CPW Disposition
    "Disposition Date" :"All",
    "LOB": ["Medicare", "Commercial"],
    "Decision Status": ["Never presented","Reject", "No Decision", "Suppress"],
    "Decision Date": "All",                         # keep all dates
    # "Recently presented": ["Never presented"]     # example
}

# ---- 4) Apply filters ----
def apply_filters(df_in: pd.DataFrame, filters: dict, col_map: dict) -> pd.DataFrame:
    df_out = df_in.copy()

    for friendly_col, chosen in (filters or {}).items():
        actual_col = col_map.get(friendly_col, friendly_col)
        if actual_col not in df_out.columns:
            continue

        # skip filter if chosen is None / "All" / empty list
        if chosen is None:
            continue
        if isinstance(chosen, str) and chosen.strip().lower() == "all":
            continue
        if isinstance(chosen, (list, tuple, set)) and len(chosen) == 0:
            continue

        # Special handling for dates (single date, list of dates, or range dict)
        if friendly_col.lower() in ["decision date"] or "date" in friendly_col.lower():
            # If user passes range dict like {"from":"2024-01-01","to":"2024-12-31"}
            if isinstance(chosen, dict):
                dt = pd.to_datetime(df_out[actual_col], errors="coerce")
                start = pd.to_datetime(chosen.get("from"), errors="coerce")
                end = pd.to_datetime(chosen.get("to"), errors="coerce")

                if pd.notna(start):
                    df_out = df_out.loc[dt >= start]
                if pd.notna(end):
                    df_out = df_out.loc[dt <= end]
            else:
                # exact match on string representation (safe for dropdown-selected exact values)
                s = df_out[actual_col].astype(str).str.strip()
                if isinstance(chosen, (list, tuple, set)):
                    chosen_set = {str(x).strip() for x in chosen}
                    df_out = df_out.loc[s.isin(chosen_set)]
                else:
                    df_out = df_out.loc[s.eq(str(chosen).strip())]

        else:
            # normal categorical filter
            s = df_out[actual_col].astype(str).str.strip()
            if isinstance(chosen, (list, tuple, set)):
                chosen_set = {str(x).strip() for x in chosen}
                df_out = df_out.loc[s.isin(chosen_set)]
            else:
                df_out = df_out.loc[s.eq(str(chosen).strip())]

    return df_out

df_f = apply_filters(base_df, selected_filters, FILTER_COL_MAP)

print("Rows before filters:", len(base_df))
print("Rows after filters: ", len(df_f))

# ---- 5) Create pivot (based on selections) ----
# These are the columns your notebook was using. Adjust if you want different grouping.
pivot_index = []
for c in ["Topic", "Policy Collection", "Dp Key","Payer Short", "Decision Status", " Decision Date", "Change Mods","CPW Client Mods","Disposition","Disposition Date", "Cpw Reason"]:
    if c in df_f.columns:
        pivot_index.append(c)

# Values to aggregate (only include those that exist)
pivot_values = []
for v in ["Annl Edits", "Annl Agg Savings"]:
    if v in df_f.columns:
        # ensure numeric
        df_f[v] = pd.to_numeric(df_f[v], errors="coerce").fillna(0)
        pivot_values.append(v)

if len(pivot_index) == 0 or len(pivot_values) == 0:
    df_pivot = pd.DataFrame()
    print("⚠️ Pivot not created: missing pivot index/value columns in df_f.")
else:
    df_pivot = (
        df_f.pivot_table(
            index=pivot_index,
            values=pivot_values,
            aggfunc="sum",
            fill_value=0
        )
        .reset_index()
    )

df_pivot.head(5)


Rows before filters: 1550
Rows after filters:  272


,Topic,Policy Collection,Dp Key,Payer Short,Decision Status,Change Mods,CPW Client Mods,Disposition,Cpw Reason,Annl Agg Savings,Annl Edits
0,"Alpha 1-Proteinase Inhibitor (J0256, J0257)",All indications,6506,HIGAC,Reject,,3/10/20 - Please remove all Highmark LOB and c...,Invalid,"[""Performed by Another Vendor""]",2608,3
1,"Alpha 1-Proteinase Inhibitor (J0256, J0257)",All indications,6506,HIGPA,Reject,,3/10/20 - Please remove all Highmark LOB and c...,Invalid,"[""Performed by Another Vendor""]",252405,201
2,"Alpha 1-Proteinase Inhibitor (J0256, J0257)",Dual diagnosis,11674,HIGAC,Reject,,3/10/20 - Please remove all Highmark LOB and c...,Invalid,"[""Performed by Another Vendor""]",189607,194
3,"Alpha 1-Proteinase Inhibitor (J0256, J0257)",Dual diagnosis,11674,HIGDE,Reject,,3/10/20 - Please remove all Highmark LOB and c...,Invalid,"[""Performed by Another Vendor""]",34614,49
4,"Alpha 1-Proteinase Inhibitor (J0256, J0257)",Dual diagnosis,11674,HIGPA,Reject,,3/10/20 - Please remove all Highmark LOB and c...,Invalid,"[""Performed by Another Vendor""]",716739,1184


In [7]:

# ---------- Build a payer-agnostic pivot (Topic + DP Key) ----------
edits_col  = "Sum of Annl Edits" if "Sum of Annl Edits" in df_pivot.columns else "Annl Edits"
sav_col    = "Sum of Annl Agg Savings" if "Sum of Annl Agg Savings" in df_pivot.columns else "Annl Agg Savings"

df_piv2 = (
    df_pivot.groupby(["Topic", "Dp Key"], as_index=False)[[edits_col, sav_col]]
              .sum()
              .rename(columns={edits_col: "Annl Edits", sav_col: "Annl Agg Savings"})
)

# ---------- Topic_Dp_Count (drives <=10 vs >10 logic) ----------
tdc = pd.read_excel("Topic_Dp_Count.xlsx", engine="openpyxl")
tdc.columns = [c.strip() for c in tdc.columns]  # safety
tdc = tdc.rename(columns={"Dp key": "Dp Key"})  # matches your screenshot
tdc["Dp Key"] = pd.to_numeric(tdc["Dp Key"], errors="coerce")
tdc["Topic_Dp_Count"] = pd.to_numeric(tdc["Topic_Dp_Count"], errors="coerce")

topic_counts = tdc.groupby("Topic", as_index=False)["Topic_Dp_Count"].max()
small_topics = set(topic_counts.loc[topic_counts["Topic_Dp_Count"] <= 10, "Topic"])
large_topics = set(topic_counts.loc[topic_counts["Topic_Dp_Count"] > 10, "Topic"])

# ---------- Decision Date lookup for comments (from filtered base df_f) ----------
def _pick_decision_date(s: pd.Series) -> str:
    vals = s.dropna().astype(str).str.strip()
    vals = vals[vals.ne("")]
    if vals.empty:
        return "Never presented"
    if (vals.str.lower().str.contains("never")).any():
        return "Never presented"
    dt = pd.to_datetime(vals, format="%b-%Y", errors="coerce")
    if dt.notna().any():
        return dt.max().strftime("%b-%Y")
    return vals.iloc[0]

dec_map = (
    df_f.assign(**{"Dp Key": pd.to_numeric(df_f["Dp Key"], errors="coerce")})
       .groupby(["Topic", "Dp Key"], as_index=False)["Decision Date"]
       .agg(_pick_decision_date)
       .rename(columns={"Decision Date": "Comments"})
)


In [8]:
# ============================================================
# FULL STEP: Build df_to_present (AND include extra columns)
# Final df_to_present must include:
# "Decision Status", "Decision Date", "Change Mods","CPW Client Mods",
# "Disposition","Disposition Date", "Cpw Reason"
#
# NOTE (per your changes):
# 1) Use "Disposition Date" column (fallback to "Disposition Capt Date" only if needed)
# 2) Do NOT replace Decision Date with Disposition Date (keep both separate)
#    Only use a sort key to choose the latest row per (Topic, Dp Key)
# ============================================================

import pandas as pd
from openpyxl import load_workbook

# ---------- helper: read .xlsx using openpyxl (safe for pandas==0.25.3) ----------
def read_xlsx_openpyxl(path, sheet_name=0):
    wb = load_workbook(path, data_only=True)
    ws = wb.worksheets[sheet_name] if isinstance(sheet_name, int) else wb[sheet_name]
    rows = list(ws.values)
    if not rows or len(rows) < 2:
        return pd.DataFrame()
    header = [str(x).strip() if x is not None else "" for x in rows[0]]
    data = rows[1:]
    df_out = pd.DataFrame(data, columns=header)
    keep_cols_mask = [str(c).strip() != "" for c in df_out.columns]  # pandas 0.25 safe
    df_out = df_out.loc[:, keep_cols_mask]
    df_out.columns = [str(c).strip() for c in df_out.columns]
    return df_out

# ------------------------------------------------------------
# 0) Topic+DP pivot (payer-agnostic) from df_pivot
# ------------------------------------------------------------
edits_col = "Sum of Annl Edits" if "Sum of Annl Edits" in df_pivot.columns else "Annl Edits"
sav_col   = "Sum of Annl Agg Savings" if "Sum of Annl Agg Savings" in df_pivot.columns else "Annl Agg Savings"

df_piv2 = (
    df_pivot.groupby(["Topic", "Dp Key"], as_index=False)[[edits_col, sav_col]]
            .sum()
            .rename(columns={edits_col: "Annl Edits", sav_col: "Annl Agg Savings"})
)

df_piv2["Annl Edits"] = pd.to_numeric(df_piv2["Annl Edits"], errors="coerce").fillna(0)
df_piv2["Annl Agg Savings"] = pd.to_numeric(df_piv2["Annl Agg Savings"], errors="coerce").fillna(0)

# ------------------------------------------------------------
# 1) Read Topic_Dp_Count (drives <=10 vs >10 logic)
# ------------------------------------------------------------
tdc = read_xlsx_openpyxl("Topic_Dp_Count.xlsx", sheet_name=0)
tdc.columns = [str(c).strip() for c in tdc.columns]
if "Dp key" in tdc.columns and "Dp Key" not in tdc.columns:
    tdc = tdc.rename(columns={"Dp key": "Dp Key"})

tdc["Dp Key"] = pd.to_numeric(tdc["Dp Key"], errors="coerce")
tdc["Topic_Dp_Count"] = pd.to_numeric(tdc["Topic_Dp_Count"], errors="coerce")

topic_cnt = tdc.groupby("Topic", as_index=False)["Topic_Dp_Count"].max()
topic_cnt_map = dict(zip(topic_cnt["Topic"], topic_cnt["Topic_Dp_Count"]))

# qualify small topics: <=10 AND must have >=1 active DP
active_topic = set(
    df_piv2.loc[(df_piv2["Annl Edits"] > 0) | (df_piv2["Annl Agg Savings"] > 0), "Topic"].unique()
)
small_topics_ok = set(
    topic_cnt.loc[(topic_cnt["Topic_Dp_Count"] <= 10) & (topic_cnt["Topic"].isin(active_topic)), "Topic"]
)
large_topics = set(topic_cnt.loc[topic_cnt["Topic_Dp_Count"] > 10, "Topic"])

# ------------------------------------------------------------
# 2) Latest info per (Topic, Dp Key) from df_f
#    Keep Decision Date and Disposition Date separate
#    Use Disposition Date (fallback to Disposition Capt Date)
# ------------------------------------------------------------
df_f2 = df_f.copy()
df_f2["Dp Key"] = pd.to_numeric(df_f2["Dp Key"], errors="coerce")

# Parse decision date for comments + present logic (expected format "Jan-2024")
df_f2["_dd_dt"] = pd.to_datetime(
    df_f2["Decision Date"].astype(str).str.strip(),
    format="%b-%Y",
    errors="coerce"
)

# Use Disposition Date (fallback to Disposition Capt Date only if needed)
if "Disposition Date" in df_f2.columns:
    df_f2["_disp_dt"] = pd.to_datetime(df_f2["Disposition Date"], errors="coerce")
elif "Disposition Capt Date" in df_f2.columns:
    df_f2["_disp_dt"] = pd.to_datetime(df_f2["Disposition Capt Date"], errors="coerce")
else:
    df_f2["_disp_dt"] = pd.NaT

# Sort key ONLY for picking latest row; does NOT overwrite decision date column
df_f2["_sort_dt"] = df_f2["_dd_dt"].where(df_f2["_dd_dt"].notna(), df_f2["_disp_dt"])

latest_idx = (
    df_f2.sort_values(["Topic", "Dp Key", "_sort_dt"])
         .groupby(["Topic", "Dp Key"], as_index=False)
         .tail(1)
         .index
)

# Pull latest fields (keep BOTH Decision Date and Disposition Date)
cols_needed = [
    "Topic", "Dp Key",
    "Decision Status", "Decision Date",
    "Change Mods", "CPW Client Mods",
    "CPW Disp", "CPW Reason"
]

# Add whichever disposition-date column exists
if "Disposition Date" in df_f2.columns:
    cols_needed.append("Disposition Date")
elif "Disposition Capt Date" in df_f2.columns:
    cols_needed.append("Disposition Capt Date")

latest_info = df_f2.loc[latest_idx, cols_needed + ["_dd_dt"]].copy()

# Standardize naming
latest_info = latest_info.rename(columns={
    "CPW Disp": "Disposition",
    "CPW Reason": "Cpw Reason",
    "_dd_dt": "DecisionDate_dt"
})

# Normalize Disposition Date column name
if "Disposition Capt Date" in latest_info.columns and "Disposition Date" not in latest_info.columns:
    latest_info = latest_info.rename(columns={"Disposition Capt Date": "Disposition Date"})

# ------------------------------------------------------------
# 3) Comments logic (uses latest decision status/date)
# ------------------------------------------------------------
latest_info["DecisionYear"] = latest_info["DecisionDate_dt"].dt.year

def _comment_from_status(status, year, dt):
    s = "" if pd.isna(status) else str(status).strip()
    if pd.isna(dt):
        return "Never presented"
    sl = s.lower()
    if sl == "no decision":
        return "No Decision in ({})".format(int(year)) if pd.notna(year) else "No Decision"
    if sl == "reject":
        return "Previously Rejected in ({})".format(int(year)) if pd.notna(year) else "Previously Rejected"
    if sl == "suppress":
        return "Previously Suppressed in ({})".format(int(year)) if pd.notna(year) else "Previously Suppressed"
    return dt.strftime("%b-%Y")

latest_info["Comments_auto"] = latest_info.apply(
    lambda r: _comment_from_status(r["Decision Status"], r["DecisionYear"], r["DecisionDate_dt"]),
    axis=1
)

# ------------------------------------------------------------
# 4A) <=10 topics (ONLY those qualifying): include ALL DP keys -> "To complete topic"
#     Still attach latest_info fields where present
# ------------------------------------------------------------
df_small = (
    tdc.loc[tdc["Topic"].isin(small_topics_ok), ["Topic", "Dp Key"]]
       .dropna(subset=["Dp Key"])
       .drop_duplicates()
       .merge(df_piv2, on=["Topic", "Dp Key"], how="left")
       .merge(latest_info.drop(columns=["DecisionYear"]), on=["Topic", "Dp Key"], how="left")
)

df_small["Annl Edits"] = pd.to_numeric(df_small["Annl Edits"], errors="coerce").fillna(0).astype("int64")
df_small["Annl Agg Savings"] = pd.to_numeric(df_small["Annl Agg Savings"], errors="coerce").fillna(0).astype("int64")

# Override comments for small topics
df_small["Comments"] = "To complete topic"

# ------------------------------------------------------------
# 4B) >10 topics: include ONLY active DPs -> comments from status/date logic
# ------------------------------------------------------------
df_large = (
    df_piv2.loc[
        df_piv2["Topic"].isin(large_topics) &
        ((df_piv2["Annl Edits"] > 0) | (df_piv2["Annl Agg Savings"] > 0))
    ]
    .merge(latest_info.drop(columns=["DecisionYear"]), on=["Topic", "Dp Key"], how="left")
)

df_large["Annl Edits"] = pd.to_numeric(df_large["Annl Edits"], errors="coerce").fillna(0).astype("int64")
df_large["Annl Agg Savings"] = pd.to_numeric(df_large["Annl Agg Savings"], errors="coerce").fillna(0).astype("int64")
df_large["Comments"] = df_large["Comments_auto"].fillna("Never presented")

# ------------------------------------------------------------
# 5) Combine
# ------------------------------------------------------------
df_to_present = pd.concat([df_small, df_large], ignore_index=True)

# ------------------------------------------------------------
# 6) Add Policy Collection (join on DP Key)
# ------------------------------------------------------------
pc_map = read_xlsx_openpyxl("Policy Collection Mapping.xlsx", sheet_name=0)
pc_map.columns = [str(c).strip() for c in pc_map.columns]

if "DPKey" not in pc_map.columns and "Dp Key" in pc_map.columns:
    pc_map = pc_map.rename(columns={"Dp Key": "DPKey"})
if "PolicyCollection" not in pc_map.columns and "Policy Collection" in pc_map.columns:
    pc_map = pc_map.rename(columns={"Policy Collection": "PolicyCollection"})

pc_map = pc_map[["DPKey", "PolicyCollection"]].drop_duplicates()
pc_map["DPKey"] = pd.to_numeric(pc_map["DPKey"], errors="coerce")

df_to_present = (
    df_to_present.merge(pc_map, how="left", left_on="Dp Key", right_on="DPKey")
                 .drop(columns=["DPKey"])
                 .rename(columns={"PolicyCollection": "Policy Collection"})
)

# ------------------------------------------------------------
# 7) Completing Topic
# ------------------------------------------------------------
def completing_topic_label(topic, comment):
    if str(comment).strip().lower() == "to complete topic":
        return "YES"
    n = topic_cnt_map.get(topic, None)
    if n is None or n <= 10:
        return "NO"
    threshold = ((int(n) - 1) // 5) * 5
    return "NO- Too many DPs ({}+)".format(threshold)

df_to_present["Completing Topic"] = df_to_present.apply(
    lambda r: completing_topic_label(r["Topic"], r["Comments"]),
    axis=1
)

# ------------------------------------------------------------
# 8) Present column (Never presented OR To complete topic OR Decision Date older than 24 months)
# ------------------------------------------------------------
today = pd.Timestamp.today().normalize()
cutoff = today - pd.DateOffset(months=24)

df_to_present["Present"] = "NO"
c_lower = df_to_present["Comments"].astype(str).str.strip().str.lower()

df_to_present.loc[c_lower.isin(["never presented", "to complete topic"]), "Present"] = "YES"
df_to_present.loc[
    df_to_present["DecisionDate_dt"].notna() & (df_to_present["DecisionDate_dt"] <= cutoff),
    "Present"
] = "YES"

# ------------------------------------------------------------
# 9) Final schema: ensure requested columns exist
# ------------------------------------------------------------
for col in ["Decision Status", "Decision Date", "Change Mods", "CPW Client Mods",
            "Disposition", "Disposition Date", "Cpw Reason"]:
    if col not in df_to_present.columns:
        df_to_present[col] = None

# Final column order
df_to_present = df_to_present[
    [
        "Topic",
        "Policy Collection",
        "Dp Key",
        "Decision Status",
        "Decision Date",
        "Change Mods",
        "CPW Client Mods",
        "Disposition",
        "Disposition Date",
        "Cpw Reason",
        "Present",
        "Comments",
        "Completing Topic",
        "Annl Edits",
        "Annl Agg Savings",
    ]
]

df_to_present.head(20)


,Topic,Policy Collection,Dp Key,Decision Status,Decision Date,Change Mods,CPW Client Mods,Disposition,Disposition Date,Cpw Reason,Present,Comments,Completing Topic,Annl Edits,Annl Agg Savings
0,Drug Administration for Drugs and Biologicals,NaN,6465,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YES,To complete topic,YES,0,0
1,Drug Administration for Drugs and Biologicals,NaN,6464,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YES,To complete topic,YES,0,0
2,Drug Administration for Drugs and Biologicals,Route of administration,7203,Reject,Dec-2022,,11/19: Client requests off cycle release to re...,Invalid,2025-07-11,"[""Conflicts with Client's Benefit or Internal ...",YES,To complete topic,YES,8833,306697
3,"Immune Globulins, IV (90283, J1459, J1552, J15...",All indications,7850,Reject,Dec-2022,,"1/23: Remove from DP, with HMK rationale: ""Ple...",Invalid,2025-07-11,"[""Performed by Another Vendor""]",YES,To complete topic,YES,1154,2320928
4,"Immune Globulins, IV (90283, J1459, J1552, J15...",NaN,7838,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YES,To complete topic,YES,0,0
5,"Immune Globulins, IV (90283, J1459, J1552, J15...",Lymphoma in remission,18331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YES,To complete topic,YES,0,0
6,"Immune Globulins, IV (90283, J1459, J1552, J15...",Age,7839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YES,To complete topic,YES,0,0
7,"Immune Globulins, IV (90283, J1459, J1552, J15...",NaN,14761,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YES,To complete topic,YES,0,0
8,"Immune Globulins, IV (90283, J1459, J1552, J15...",Dual diagnosis,14762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YES,To complete topic,YES,0,0
9,"Immune Globulins, IV (90283, J1459, J1552, J15...",NaN,14764,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YES,To complete topic,YES,0,0


In [9]:
df_to_present.to_excel("df_to_present.xlsx", index=False)